In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Sample dataset of movie ratings by users
data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4],
    'movie_id': [101, 102, 103, 101, 103, 104, 101, 104, 102, 103],
    'rating': [5, 4, 3, 5, 4, 2, 3, 5, 4, 3]
}

# Convert data into a Pandas DataFrame
df = pd.DataFrame(data)

# Create a pivot table where rows are users and columns are movies
user_movie_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')

# Fill NaN values with 0 (movies that haven't been rated by a user)
user_movie_matrix.fillna(0, inplace=True)

# Compute the cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)

# Convert to a DataFrame for better readability
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to get movie recommendations for a user based on user-user collaborative filtering
def get_recommendations(user_id, num_recommendations=2):
    # Find similar users to the input user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # Exclude the user themselves

    # Get the movies watched by the similar users
    watched_by_similar_users = user_movie_matrix.loc[similar_users.index]
    
    # Calculate the weighted sum of ratings based on similarity scores
    weighted_ratings = np.dot(similar_users.values, watched_by_similar_users)
    
    # Create a recommendation score for each movie by averaging ratings from similar users
    recommendation_scores = pd.Series(weighted_ratings, index=user_movie_matrix.columns)
    
    # Remove movies the user has already watched
    watched_movies = user_movie_matrix.loc[user_id]
    recommendation_scores = recommendation_scores[watched_movies == 0]
    
    # Return the top N recommendations
    return recommendation_scores.nlargest(num_recommendations)

# Example usage
user_id = 1  # Recommend movies for user 1
recommendations = get_recommendations(user_id)
print(f"Top movie recommendations for user {user_id}:")
print(recommendations)


Top movie recommendations for user 1:
movie_id
104    3.379074
dtype: float64
